In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import exp


# stochastic gradient descent (SGD)
using batches to fasten gradient computation (approximation)

SGD has to be implemented with static stepsize

In [ ]:
def sigmoid(x):
    return 1/(1-exp(-x))

def f(w, xhat):
    return sigmoid(xhat.T*w)

def MSE(y, y1):
    return (np.linalg.norm(y-y1)**2)/y.size[0]

def l(w,X, Y): # D is dataset made of X data and Y labels, w is the weights array
    return (MSE(f(w,X), Y))



def grad(w, D):
    #D is the current batch: a tuple (Mx, My)
    #gradient of SGD (sommatoria dei gradienti degli indici del batch [subset M])
    #this is an approximation of the gradiant, it isn't for sure the direction of steepest descent
    return sigmmoid()

def SGD(l, grad_l, w0, D, batch_size, n_epochs):
    #same outputs as GD
    #initialization
    a=1 #fixed
    # D =(X,Y) where X is d x N 
    # Y is N
    X, Y= D
    d,N=X.shape
    n_batch_per_epoch=int(N/batch_size)
    #iterations
    k=0
    f_val[0]=x0
    conditions=True
    X_backup=X
    Y_backup=Y
    #loop
    for epoch in range(n_epochs):
        shuffledInd=np.random.shuffle(np.arange(Y.shape[0])) #shuffle indici

        for k in range (n_batch_per_epoch):
            #sample M from D
            batchInd=shuffledInd[n_batch_per_epoch*k : n_batch_per_epoch*(k+1)]
            Mx=X[:,batchInd] #batch di X
            My=Y[batchInd]#batch di Y
            M=(Mx, My)
            #remove Mx and My from X and Y
            del X[,batchInd]
            del Y[batchInd]
            #update w
            w=w0-a*grad(M)
            f_val[k]=l(w, M)
            #restart 
            w0=w
        #reload X and Y
        X=X_backup
        Y=Y_backup
        #NOTA: YOU HAVE TO SHUFFLE AGAIN AFTER EACH EPOCH!!!! 
        #so we can sample each time different batches
        #reshuffle
    return w
 #REMEMBER: w0 in SGD should be chosen randomly (sample from Gaussian), and not fixed
 #(upgrade): modify the code above to return f_val (over M)
 #f_val[k]=l(w_k, M)
 
